In [1]:
# "Sanaz Hosseini"- University of North Carolina at Charlotte
# Introduction to Machine Learning Class - Instructor: Prof. Hamed Tabkhi
# Fully Connected Neural Networks VS Convolutional Neural Network
# Homework 6

In [2]:
import torch
import time
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
%matplotlib inline

In [11]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


Files already downloaded and verified


In [12]:
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [14]:
import torch.nn as nn

n_out = 10

model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.ReLU(),
            nn.Linear(512, n_out)
        )

In [17]:
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 10),
            nn.Softmax(dim=1))

In [18]:
loss = nn.NLLLoss()

In [20]:
imgs, label = cifar10[0]

out = model(img.view(-1).unsqueeze(0))

loss(out, torch.tensor([label]))

tensor(-0.0785, grad_fn=<NllLossBackward0>)

In [21]:
import datetime  
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.LogSoftmax(dim=1))

model.to(device)
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()
n_epochs = 300

for epoch in range(1, n_epochs + 1): 
        loss_train = 0.0
        training_start_time = time.time()
        for imgs, labels in train_loader:  
            imgs=imgs.to(device)
            labels=labels.to(device)
            outputs = model(imgs.view(imgs.shape[0], -1))
            loss = loss_fn(outputs, labels) 
            
            optimizer.zero_grad() 
            loss.backward()  
            optimizer.step()  
            loss_train += loss.item() 
            
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))  

2022-12-10 13:45:07.376976 Epoch 1, Training loss 1.7207147134539416
2022-12-10 13:50:25.869525 Epoch 10, Training loss 1.0912875848657944
2022-12-10 13:58:16.715891 Epoch 20, Training loss 0.7841284063923389
2022-12-10 14:06:47.623993 Epoch 30, Training loss 0.5789946576251703
2022-12-10 14:16:00.761149 Epoch 40, Training loss 0.4558614944024464
2022-12-10 14:24:58.874136 Epoch 50, Training loss 0.41936354565403194
2022-12-10 14:33:41.515398 Epoch 60, Training loss 0.38958945333638495
2022-12-10 14:42:01.329633 Epoch 70, Training loss 0.3121620452607913
2022-12-10 14:49:08.410329 Epoch 80, Training loss 0.32396783360191966
2022-12-10 14:56:13.827367 Epoch 90, Training loss 0.29715627053623916
2022-12-10 15:04:02.463644 Epoch 100, Training loss 0.32481246206300846
2022-12-10 15:12:07.430403 Epoch 110, Training loss 0.30454796869480444
2022-12-10 15:20:36.741313 Epoch 120, Training loss 0.2859015967421498
2022-12-10 15:27:43.439637 Epoch 130, Training loss 0.2919010809258572
2022-12-10 

In [22]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs=imgs.to(device)
        labels=labels.to(device)
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 0.967940
